In [57]:
import numpy as np
import pandas as pd

In [58]:
# vars
crime_data_filepath = "C:/Users/mattd/Downloads/NYPD_Complaint_Data_Historic_20240904.csv"
min_dt = '1900-01-01'
max_dt = '2199-12-31'
min_tm = '00:00:00'
max_tm = '23:59:59'
min_x_coord = '0'
max_x_coord = '10000000'
min_y_coord = '0'
max_y_coord = '10000000'

In [59]:
def check_bounds(items, min, max):
    # force string conversion for comparison purposes
    items = [str(item) for item in items]

    # return if in bounds in order of items
    for item in items:
        if(item >= min and item <= max):
            return item
    
    # otherwise, return the min value
    return min

In [60]:
# list of columns to load in
cols_load = [
    'CMPLNT_NUM',
    'CMPLNT_FR_DT',
    'CMPLNT_FR_TM',
    'CMPLNT_TO_DT',
    'CMPLNT_TO_TM',
    'ADDR_PCT_CD',
    'PATROL_BORO',
    'KY_CD',
    'PD_CD',
    'PREM_TYP_DESC',
    'LOC_OF_OCCUR_DESC',
    'X_COORD_CD',
    'Y_COORD_CD'
]

# dict of column names and their new names
col_rename_map = {
    'CMPLNT_NUM': 'cmplnt_num',
    'CMPLNT_FR_DT': 'cmplnt_fr_dt',
    'CMPLNT_FR_TM': 'cmplnt_fr_tm',
    'CMPLNT_TO_DT': 'cmplnt_to_dt',
    'CMPLNT_TO_TM': 'cmplnt_to_tm',
    'ADDR_PCT_CD': 'pct',
    'PATROL_BORO': 'boro',
    'KY_CD': 'ky_cd',
    'PD_CD': 'pd_cd',
    'PREM_TYP_DESC': 'prem_typ_desc',
    'LOC_OF_OCCUR_DESC': 'loc_occur_desc',
    'X_COORD_CD': 'x_coord',
    'Y_COORD_CD': 'y_coord'
}

# list of columns that are required
cols_drop_any = [
    'cmplnt_num',
    'x_coord',
    'y_coord'
]

# list of lists of columns where at least one within the list is required
cols_drop_all = [
    [
        'cmplnt_fr_dt',
        'cmplnt_to_dt'
    ],
    [
        'cmplnt_fr_tm',
        'cmplnt_to_tm'
    ]
]

# list of date and time columns
cols_datetime = [
    'cmplnt_fr_dt',
    'cmplnt_fr_tm',
    'cmplnt_to_dt',
    'cmplnt_to_tm'
]

# map of patrol boro names to shortened versions
patrol_boro_rename_map = {
    'PATROL BORO BKLYN NORTH': 'PBBN',
    'PATROL BORO BKLYN SOUTH': 'PBBS',
    'PATROL BORO MAN NORTH': 'PBMN',
    'PATROL BORO MAN SOUTH': 'PBMS',
    'PATROL BORO QUEENS NORTH': 'PBQN',
    'PATROL BORO QUEENS SOUTH': 'PBQS',
    'PATROL BORO BRONX': 'PBBX',
    'PATROL BORO STATEN ISLAND': 'PBSI'
}

In [61]:
# load data from csv into dataframe
crimes_df = pd.read_csv(crime_data_filepath, usecols=cols_load)

C:\Users\mattd\AppData\Local\Temp\ipykernel_808\3107796073.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  crimes_df = pd.read_csv(crime_data_filepath, usecols=cols_load)


In [62]:
# rename columns
crimes_df = crimes_df.rename(columns=col_rename_map)

In [63]:
# convert date and time columns
crimes_df['cmplnt_fr_dt'] = pd.to_datetime(crimes_df.cmplnt_fr_dt, errors='coerce')
crimes_df['cmplnt_fr_tm'] = pd.to_timedelta(crimes_df.cmplnt_fr_tm, errors='coerce')
crimes_df['cmplnt_to_dt'] = pd.to_datetime(crimes_df.cmplnt_to_dt, errors='coerce')
crimes_df['cmplnt_to_tm'] = pd.to_timedelta(crimes_df.cmplnt_to_tm, errors='coerce')

In [64]:
# drop record if any of these columns are null
crimes_df = crimes_df.dropna(subset=cols_drop_any, how='any')

In [65]:
# drop record if all of the columns in each list is null
for cols in cols_drop_all:
    crimes_df = crimes_df.dropna(subset=cols, how='all')

In [66]:
# keep last from duplicates of cmplnt_num
crimes_df = crimes_df.drop_duplicates(subset='cmplnt_num', keep='last')

In [67]:
# drop date and time columns as they are no longer needed
# crimes_df = crimes_df.drop(columns=cols_datetime)

In [68]:
# rename boro to shortened versions
crimes_df['boro'] = crimes_df['boro'].map(patrol_boro_rename_map)

In [69]:
# reset index
crimes_df = crimes_df.reset_index()

In [70]:
crimes_df

,index,cmplnt_num,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_dt,cmplnt_to_tm,pct,ky_cd,pd_cd,loc_occur_desc,prem_typ_desc,x_coord,y_coord,boro
0,0,39468181,2008-02-20,0 days 07:00:00,2008-02-23,0 days 08:00:00,88.0,107,221.0,INSIDE,RESIDENCE - APT. HOUSE,991818.0,191560.0,PBBN
1,1,50539499,2008-08-21,0 days 22:00:00,2008-08-21,0 days 23:00:00,19.0,109,438.0,FRONT OF,BAR/NIGHT CLUB,997152.0,220300.0,PBMN
2,2,45223390,2008-04-03,0 days 03:35:00,2008-04-03,0 days 03:50:00,77.0,106,109.0,INSIDE,RESIDENCE - PUBLIC HOUSING,1004579.0,183837.0,PBBN
3,3,50594658,2008-08-19,0 days 09:00:00,NaT,NaT,32.0,341,349.0,FRONT OF,STREET,999965.0,235630.0,PBMN
4,4,44451016,2008-03-10,0 days 22:00:00,2008-03-10,0 days 22:10:00,67.0,105,397.0,FRONT OF,STREET,999602.0,176145.0,PBBS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913118,8914833,47410177,2008-06-21,0 days 00:03:00,2008-06-21,0 days 00:49:00,72.0,344,101.0,(null),STREET,991830.0,177052.0,PBBS
8913119,8914834,45722549,2008-04-16,0 days 13:47:00,NaT,NaT,30.0,113,729.0,INSIDE,DEPARTMENT STORE,998031.0,240493.0,PBMN
8913120,8914835,49520303,2008-07-30,0 days 01:15:00,2008-07-30,0 days 01:28:00,34.0,235,511.0,(null),PARK/PLAYGROUND,1004500.0,250318.0,PBMN
8913121,8914836,47521554,2008-06-25,0 days 14:40:00,2008-06-25,0 days 15:00:00,60.0,344,101.0,FRONT OF,RESIDENCE - APT. HOUSE,991938.0,149951.0,PBBS


In [88]:
zone = crimes_df.query('''cmplnt_fr_dt >= '2023-05-01' and cmplnt_fr_dt <= '2023-09-18' and x_coord >= 1023456.509864 and x_coord <= 1046482.056801 and y_coord >= 215172.268685 and y_coord <= 215851.260787''')

In [93]:
counts = zone.value_counts(subset='ky_cd')

In [94]:
total = len(zone)

In [104]:
for i in range(10):
    print("key code: {:.0f} - {:.2f}%".format(counts.index[i], counts.iloc[i] / total * 100))

key code: 341 - 48.62%
key code: 578 - 10.53%
key code: 109 - 10.40%
key code: 344 - 8.27%
key code: 106 - 2.51%
key code: 348 - 2.38%
key code: 351 - 2.38%
key code: 107 - 2.26%
key code: 126 - 2.01%
key code: 110 - 2.01%


In [107]:
crimes_df.dtypes

index                       int64
cmplnt_num                 object
cmplnt_fr_dt       datetime64[ns]
cmplnt_fr_tm      timedelta64[ns]
cmplnt_to_dt       datetime64[ns]
cmplnt_to_tm      timedelta64[ns]
pct                       float64
ky_cd                       int64
pd_cd                     float64
loc_occur_desc             object
prem_typ_desc              object
x_coord                   float64
y_coord                   float64
boro                       object
dtype: object